# Computational Neuroscience Final Project
## Elizabeth Frey

In [71]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Import Data

In [85]:
import pandas as pd
import numpy as np

# read csv file
df = pd.read_csv('data-en-hi-de-fr.csv')

In [90]:
from tensorflow.keras.utils import to_categorical

# group the dataframe by labels column
grouped = df.groupby('labels')

# create two dataframes, one for ham and one for spam
ham_df = grouped.get_group('ham')
spam_df = grouped.get_group('spam')
ham_df = ham_df.drop('labels', axis=1)
spam_df = spam_df.drop('labels', axis=1)

# pivot the data so that each row is a phrase in a language
ham_df = ham_df.rename(
    columns={'text': 'en', 'text_hi': 'hi', 'text_de': 'de', 'text_fr': 'fr'})
ham_df = ham_df.melt(var_name='language', value_name='phrase')
spam_df = spam_df.rename(
    columns={'text': 'en', 'text_hi': 'hi', 'text_de': 'de', 'text_fr': 'fr'})
spam_df = spam_df.melt(var_name='language', value_name='phrase')

# create a dictionary of the languages
languages = {'en': 0, 'hi': 1, 'de': 2, 'fr': 3}

# convert the languages to numbers
ham_df['language_code'] = ham_df['language'].map(languages)
spam_df['language_code'] = spam_df['language'].map(languages)

# one hot encode the languages using to categorical
# this is taken from project part 1


def one_hot(i):
    one_hot_encoded = np.array([0]*4)
    one_hot_encoded[i-1] = 1
    return np.array(one_hot_encoded)


ham_df['language_one_hot'] = ham_df['language_code'].apply(one_hot)
spam_df['language_one_hot'] = spam_df['language_code'].apply(one_hot)

## Embeddings

Creates functions to test out differente embeddings.

In [92]:
# define a function to tokenize the phrases in each dataframe


def tokenize(df):
    return [phrase.split() for phrase in df['phrase']]


# tokenize the phrases in each dataframe
ham_tokens = tokenize(ham_df)
spam_tokens = tokenize(spam_df)

ham_df['tokens'] = ham_tokens
spam_df['tokens'] = spam_tokens

### Word2Vec Embeddings

In [94]:
import gensim
from gensim.models import Word2Vec


# train the word2vec model on the ham and spam tokens
model = Word2Vec(ham_tokens + spam_tokens, min_count=1)

# create the embeddings for each dataframe
ham_df['w2v_embedding'] = ham_df['tokens'].apply(
    lambda x: model.wv[x].mean(axis=0).astype(np.float32))
spam_df['w2v_embedding'] = spam_df['tokens'].apply(
    lambda x: model.wv[x].mean(axis=0).astype(np.float32))

## Train the Model

In [110]:
# create training and testing datasets from the ham and spam dataframes
from sklearn.model_selection import train_test_split

X = pd.concat([ham_df['w2v_embedding'], spam_df['w2v_embedding']])
y = pd.concat([ham_df['language_one_hot'], spam_df['language_one_hot']])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [106]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential()
model.add(layers.GRU(64, input_shape=(28, 28)))
model.add(layers.BatchNormalization())
model.add(layers.Dense(10))
print(model.summary())

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 64)                18048     
                                                                 
 batch_normalization_4 (Bat  (None, 64)                256       
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, 10)                650       
                                                                 
Total params: 18954 (74.04 KB)
Trainable params: 18826 (73.54 KB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________
None


In [109]:
model.fit(
    X_train, y_train, batch_size=64, epochs=10
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).